# QQQ 단기 매수·매도 진입 타이밍 감지 시스템

이 노트북은 QQQ 1분봉 차트에서 SQZMOM_LB (TTM Squeeze Momentum)와 Stochastic RSI 지표를 활용하여 단기 매수·매도 진입 타이밍을 자동으로 감지하고 시각화합니다.

## 주요 기능
- TTM Squeeze Momentum 지표 계산
- Stochastic RSI 지표 계산
- 매수·매도 신호 자동 생성
- 실시간 차트 시각화
- 백테스팅 및 성과 분석


## 1. 환경 설정 및 라이브러리 Import


In [ ]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (macOS)
plt.rcParams['font.family'] = 'Arial Unicode MS'
plt.rcParams['axes.unicode_minus'] = False

# 차트 스타일 설정
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = '#1e1e1e'
plt.rcParams['axes.facecolor'] = '#2d2d2d'

print("라이브러리 로딩 완료")


## 2. 데이터 로딩 및 전처리


In [ ]:
# QQQ 데이터 다운로드 (최근 5일, 1분봉)
def load_qqq_data():
    """QQQ 1분봉 데이터 로딩"""
    ticker = "QQQ"
    end_date = datetime.now()
    start_date = end_date - timedelta(days=5)
    
    # yfinance를 사용하여 데이터 다운로드
    data = yf.download(ticker, start=start_date, end=end_date, interval='1m')
    
    # 컬럼명 정리
    data.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    
    # NaN 값 제거
    data = data.dropna()
    
    print(f"데이터 로딩 완료: {len(data)}개의 1분봉 데이터")
    print(f"기간: {data.index[0]} ~ {data.index[-1]}")
    print(f"최신 가격: {data['Close'].iloc[-1]:.2f}달러")
    
    return data

# 데이터 로딩
df = load_qqq_data()
df.head()


## 3. TTM Squeeze Momentum (SQZMOM_LB) 지표 구현


In [ ]:
def calculate_squeeze_momentum(df, length=20, mult=2.0, length_kc=20, mult_kc=1.5):
    """TTM Squeeze Momentum 지표 계산"""
    
    # ATR 계산
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    atr = true_range.rolling(window=length).mean()
    
    # Bollinger Bands
    bb_middle = df['Close'].rolling(window=length).mean()
    bb_std = df['Close'].rolling(window=length).std()
    bb_upper = bb_middle + (bb_std * mult)
    bb_lower = bb_middle - (bb_std * mult)
    
    # Keltner Channels
    kc_middle = df['Close'].rolling(window=length_kc).mean()
    kc_upper = kc_middle + (atr * mult_kc)
    kc_lower = kc_middle - (atr * mult_kc)
    
    # Squeeze 감지
    squeeze_on = (bb_lower > kc_lower) & (bb_upper < kc_upper)
    squeeze_off = (bb_lower <= kc_lower) | (bb_upper >= kc_upper)
    
    # Momentum 계산
    highest = df['High'].rolling(window=length_kc).max()
    lowest = df['Low'].rolling(window=length_kc).min()
    
    avg = (highest + lowest) / 2
    mom = df['Close'] - avg
    
    # Squeeze Momentum 값
    squeeze_momentum = np.where(squeeze_on, mom, 0)
    
    return squeeze_momentum, squeeze_on, squeeze_off

# SQZMOM 계산
squeeze_mom, squeeze_on, squeeze_off = calculate_squeeze_momentum(df)
df['SQZMOM'] = squeeze_mom
df['Squeeze_On'] = squeeze_on
df['Squeeze_Off'] = squeeze_off

print("TTM Squeeze Momentum 지표 계산 완료")
print(f"최신 SQZMOM 값: {df['SQZMOM'].iloc[-1]:.4f}")


## 4. Stochastic RSI 지표 구현


In [ ]:
def calculate_stochastic_rsi(df, k_period=3, d_period=3, rsi_period=14, stoch_period=14):
    """Stochastic RSI 지표 계산"""
    
    # RSI 계산
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=rsi_period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    
    # Stochastic RSI 계산
    rsi_min = rsi.rolling(window=stoch_period).min()
    rsi_max = rsi.rolling(window=stoch_period).max()
    stoch_rsi = 100 * (rsi - rsi_min) / (rsi_max - rsi_min)
    
    # %K와 %D 계산
    stoch_rsi_k = stoch_rsi.rolling(window=k_period).mean()
    stoch_rsi_d = stoch_rsi_k.rolling(window=d_period).mean()
    
    return stoch_rsi_k, stoch_rsi_d, rsi

# Stochastic RSI 계산
stoch_rsi_k, stoch_rsi_d, rsi = calculate_stochastic_rsi(df)
df['Stoch_RSI_K'] = stoch_rsi_k
df['Stoch_RSI_D'] = stoch_rsi_d
df['RSI'] = rsi

print("Stochastic RSI 지표 계산 완료")
print(f"최신 Stoch RSI K: {df['Stoch_RSI_K'].iloc[-1]:.2f}")
print(f"최신 Stoch RSI D: {df['Stoch_RSI_D'].iloc[-1]:.2f}")


## 5. 매수·매도 신호 생성 로직


In [ ]:
def generate_trading_signals(df):
    """매수·매도 신호 생성"""
    
    signals = pd.Series(index=df.index, data=0)  # 0: 보유, 1: 매수, -1: 매도
    signal_strength = pd.Series(index=df.index, data=0)  # 신호 강도
    
    # 매수 신호 조건
    # 1. SQZMOM이 음수에서 양수로 전환 (상승 모멘텀 시작)
    sqzmom_bullish = (df['SQZMOM'] > 0) & (df['SQZMOM'].shift(1) <= 0)
    
    # 2. Stochastic RSI가 과매도 구간(20 이하)에서 반등
    stoch_oversold_bounce = (df['Stoch_RSI_K'] > df['Stoch_RSI_D']) & \
                           (df['Stoch_RSI_K'].shift(1) <= df['Stoch_RSI_D'].shift(1)) & \
                           (df['Stoch_RSI_K'] < 30)
    
    # 3. Squeeze 해제 (변동성 확대)
    squeeze_release = df['Squeeze_Off'] & df['Squeeze_On'].shift(1)
    
    # 매도 신호 조건
    # 1. SQZMOM이 양수에서 음수로 전환 (하락 모멘텀 시작)
    sqzmom_bearish = (df['SQZMOM'] < 0) & (df['SQZMOM'].shift(1) >= 0)
    
    # 2. Stochastic RSI가 과매수 구간(80 이상)에서 하락
    stoch_overbought_fall = (df['Stoch_RSI_K'] < df['Stoch_RSI_D']) & \
                           (df['Stoch_RSI_K'].shift(1) >= df['Stoch_RSI_D'].shift(1)) & \
                           (df['Stoch_RSI_K'] > 70)
    
    # 강력한 매수 신호
    strong_buy = sqzmom_bullish & stoch_oversold_bounce & squeeze_release
    
    # 일반 매수 신호
    buy_signal = (sqzmom_bullish & stoch_oversold_bounce) | \
                (sqzmom_bullish & squeeze_release) | \
                (stoch_oversold_bounce & squeeze_release)
    
    # 강력한 매도 신호
    strong_sell = sqzmom_bearish & stoch_overbought_fall
    
    # 일반 매도 신호
    sell_signal = sqzmom_bearish | stoch_overbought_fall
    
    # 신호 할당
    signals[strong_buy] = 2  # 강력한 매수
    signals[buy_signal & ~strong_buy] = 1  # 일반 매수
    signals[strong_sell] = -2  # 강력한 매도
    signals[sell_signal & ~strong_sell] = -1  # 일반 매도
    
    # 신호 강도 계산
    signal_strength[strong_buy] = 3
    signal_strength[buy_signal & ~strong_buy] = 2
    signal_strength[strong_sell] = 3
    signal_strength[sell_signal & ~strong_sell] = 2
    
    return signals, signal_strength

# 신호 생성
df['Signal'], df['Signal_Strength'] = generate_trading_signals(df)

# 최근 신호 확인
recent_signals = df[df['Signal'] != 0].tail(10)
print("최근 매매 신호:")
for idx, row in recent_signals.iterrows():
    signal_type = "강력한 매수" if row['Signal'] == 2 else "매수" if row['Signal'] == 1 else "매도" if row['Signal'] == -1 else "강력한 매도"
    print(f"{idx.strftime('%Y-%m-%d %H:%M')}: {signal_type} (강도: {row['Signal_Strength']})")


## 6. 차트 시각화


In [ ]:
def plot_trading_chart(df, lookback_hours=6):
    """매매 신호가 포함된 차트 시각화"""
    
    # 최근 데이터만 선택 (성능을 위해)
    end_time = df.index[-1]
    start_time = end_time - timedelta(hours=lookback_hours)
    recent_df = df[df.index >= start_time].copy()
    
    # 차트 생성
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 12), 
                                       gridspec_kw={'height_ratios': [3, 1, 1]})
    
    # 1. 가격 차트 (캔들스틱)
    ax1.set_title('QQQ 1분봉 차트 + 매매 신호', fontsize=16, color='white', fontweight='bold')
    
    # 캔들스틱 그리기
    for i, (idx, row) in enumerate(recent_df.iterrows()):
        color = '#00ff88' if row['Close'] >= row['Open'] else '#ff4444'
        ax1.plot([i, i], [row['Low'], row['High']], color='gray', alpha=0.7, linewidth=0.5)
        ax1.plot([i, i], [row['Open'], row['Close']], color=color, linewidth=2)
    
    # 매매 신호 표시
    buy_signals = recent_df[recent_df['Signal'] > 0]
    sell_signals = recent_df[recent_df['Signal'] < 0]
    
    for i, (idx, row) in enumerate(recent_df.iterrows()):
        if row['Signal'] > 0:  # 매수 신호
            marker = '^' if row['Signal'] == 2 else '▲'
            size = 100 if row['Signal'] == 2 else 80
            ax1.scatter(i, row['Low'] - 0.2, marker=marker, color='#00ff88', 
                       s=size, zorder=5, edgecolors='white', linewidth=1)
        elif row['Signal'] < 0:  # 매도 신호
            marker = 'v' if row['Signal'] == -2 else '▼'
            size = 100 if row['Signal'] == -2 else 80
            ax1.scatter(i, row['High'] + 0.2, marker=marker, color='#ff4444', 
                       s=size, zorder=5, edgecolors='white', linewidth=1)
    
    ax1.set_ylabel('가격 ($)', color='white')
    ax1.grid(True, alpha=0.3)
    
    # 2. SQZMOM 지표
    ax2.set_title('TTM Squeeze Momentum (SQZMOM_LB)', color='white')
    
    # 0선 그리기
    ax2.axhline(y=0, color='white', linestyle='-', alpha=0.5)
    
    # SQZMOM 바 차트
    colors = ['#00ff88' if x >= 0 else '#ff4444' for x in recent_df['SQZMOM']]
    ax2.bar(range(len(recent_df)), recent_df['SQZMOM'], color=colors, alpha=0.7, width=0.8)
    
    ax2.set_ylabel('SQZMOM', color='white')
    ax2.grid(True, alpha=0.3)
    
    # 3. Stochastic RSI
    ax3.set_title('Stochastic RSI', color='white')
    
    # 과매수/과매도 라인
    ax3.axhline(y=80, color='red', linestyle='--', alpha=0.7, label='과매수 (80)')
    ax3.axhline(y=20, color='green', linestyle='--', alpha=0.7, label='과매도 (20)')
    
    # Stoch RSI 라인
    ax3.plot(range(len(recent_df)), recent_df['Stoch_RSI_K'], color='#00bfff', 
             linewidth=2, label='%K')
    ax3.plot(range(len(recent_df)), recent_df['Stoch_RSI_D'], color='#ff8800', 
             linewidth=2, label='%D')
    
    ax3.set_ylabel('Stoch RSI', color='white')
    ax3.set_ylim(0, 100)
    ax3.legend(loc='upper right')
    ax3.grid(True, alpha=0.3)
    
    # X축 시간 라벨 설정
    x_ticks = range(0, len(recent_df), max(1, len(recent_df)//10))
    x_labels = [recent_df.index[i].strftime('%H:%M') for i in x_ticks]
    
    for ax in [ax1, ax2, ax3]:
        ax.set_xticks(x_ticks)
        ax.set_xticklabels(x_labels, rotation=45)
        ax.tick_params(colors='white')
    
    ax3.set_xlabel('시간', color='white')
    
    plt.tight_layout()
    plt.show()
    
    # 현재 상태 출력
    latest = recent_df.iloc[-1]
    print(f"\n=== 현재 상태 ({latest.name.strftime('%Y-%m-%d %H:%M')}) ===")
    print(f"현재가: ${latest['Close']:.2f}")
    print(f"SQZMOM: {latest['SQZMOM']:.4f} {'(상승 모멘텀)' if latest['SQZMOM'] > 0 else '(하락 모멘텀)'}")
    print(f"Stoch RSI K: {latest['Stoch_RSI_K']:.2f}")
    print(f"Stoch RSI D: {latest['Stoch_RSI_D']:.2f}")
    print(f"Squeeze 상태: {'해제됨' if latest['Squeeze_Off'] else '진행중'}")
    
    if latest['Signal'] != 0:
        signal_text = "강력한 매수" if latest['Signal'] == 2 else "매수" if latest['Signal'] == 1 else "매도" if latest['Signal'] == -1 else "강력한 매도"
        print(f"매매 신호: {signal_text} (강도: {latest['Signal_Strength']})")
    else:
        print("매매 신호: 대기")

# 차트 시각화
plot_trading_chart(df, lookback_hours=6)


## 7. 실시간 모니터링 함수


In [ ]:
def monitor_realtime_signals(df, check_interval_minutes=5):
    """실시간 신호 모니터링"""
    
    # 최근 1시간 데이터만 분석
    recent_hour = df.tail(60)  # 1분봉이므로 60개 = 1시간
    
    # 최근 신호 확인
    recent_signals = recent_hour[recent_hour['Signal'] != 0]
    
    print("=== 실시간 모니터링 ===")
    print(f"모니터링 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    if len(recent_signals) > 0:
        latest_signal = recent_signals.iloc[-1]
        signal_type = "강력한 매수" if latest_signal['Signal'] == 2 else \
                     "매수" if latest_signal['Signal'] == 1 else \
                     "매도" if latest_signal['Signal'] == -1 else \
                     "강력한 매도"
        
        print(f"🚨 새로운 신호 감지! 🚨")
        print(f"신호 유형: {signal_type}")
        print(f"신호 시간: {latest_signal.name.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"가격: ${latest_signal['Close']:.2f}")
        print(f"SQZMOM: {latest_signal['SQZMOM']:.4f}")
        print(f"Stoch RSI: K={latest_signal['Stoch_RSI_K']:.2f}, D={latest_signal['Stoch_RSI_D']:.2f}")
    else:
        latest = recent_hour.iloc[-1]
        print(f"현재 상태: 대기 중")
        print(f"현재가: ${latest['Close']:.2f}")
        print(f"SQZMOM: {latest['SQZMOM']:.4f}")
        print(f"Stoch RSI: K={latest['Stoch_RSI_K']:.2f}, D={latest['Stoch_RSI_D']:.2f}")
        
        # 신호 가능성 분석
        potential_signals = []
        if latest['SQZMOM'] > -0.1 and latest['SQZMOM'] < 0.1:
            potential_signals.append("SQZMOM 전환점 근접")
        if latest['Stoch_RSI_K'] < 25:
            potential_signals.append("과매도 구간 근접")
        if latest['Stoch_RSI_K'] > 75:
            potential_signals.append("과매수 구간 근접")
        if latest['Squeeze_Off']:
            potential_signals.append("변동성 확대 중")
        
        if potential_signals:
            print("\n📊 주의 관찰 사항:")
            for signal in potential_signals:
                print(f"  • {signal}")

# 실시간 모니터링 실행
monitor_realtime_signals(df)


## 8. 백테스팅 및 성과 분석


In [ ]:
def backtest_strategy(df, initial_capital=10000):
    """백테스팅 수행"""
    
    capital = initial_capital
    position = 0  # 0: 없음, 1: 롱, -1: 숏
    entry_price = 0
    trades = []
    
    for i, (idx, row) in enumerate(df.iterrows()):
        current_price = row['Close']
        signal = row['Signal']
        
        # 매수 신호 처리
        if signal > 0 and position <= 0:  # 매수 또는 숏 청산
            if position == -1:  # 숏 청산
                pnl = (entry_price - current_price) / entry_price
                capital *= (1 + pnl)
                trades.append({
                    'entry_time': entry_time,
                    'exit_time': idx,
                    'type': 'SHORT',
                    'entry_price': entry_price,
                    'exit_price': current_price,
                    'pnl': pnl,
                    'capital': capital
                })
            
            # 롱 진입
            position = 1
            entry_price = current_price
            entry_time = idx
        
        # 매도 신호 처리
        elif signal < 0 and position >= 0:  # 매도 또는 롱 청산
            if position == 1:  # 롱 청산
                pnl = (current_price - entry_price) / entry_price
                capital *= (1 + pnl)
                trades.append({
                    'entry_time': entry_time,
                    'exit_time': idx,
                    'type': 'LONG',
                    'entry_price': entry_price,
                    'exit_price': current_price,
                    'pnl': pnl,
                    'capital': capital
                })
            
            # 숏 진입
            position = -1
            entry_price = current_price
            entry_time = idx
    
    # 마지막 포지션 청산
    if position != 0:
        current_price = df['Close'].iloc[-1]
        if position == 1:  # 롱 청산
            pnl = (current_price - entry_price) / entry_price
            capital *= (1 + pnl)
        elif position == -1:  # 숏 청산
            pnl = (entry_price - current_price) / entry_price
            capital *= (1 + pnl)
        
        trades.append({
            'entry_time': entry_time,
            'exit_time': df.index[-1],
            'type': 'LONG' if position == 1 else 'SHORT',
            'entry_price': entry_price,
            'exit_price': current_price,
            'pnl': pnl,
            'capital': capital
        })
    
    return trades, capital

# 백테스팅 실행
trades, final_capital = backtest_strategy(df)

# 성과 분석
if trades:
    trades_df = pd.DataFrame(trades)
    
    total_return = (final_capital - 10000) / 10000 * 100
    win_trades = trades_df[trades_df['pnl'] > 0]
    lose_trades = trades_df[trades_df['pnl'] <= 0]
    
    win_rate = len(win_trades) / len(trades_df) * 100
    avg_win = win_trades['pnl'].mean() * 100 if len(win_trades) > 0 else 0
    avg_loss = lose_trades['pnl'].mean() * 100 if len(lose_trades) > 0 else 0
    
    print("=== 백테스팅 결과 ===")
    print(f"초기 자본: $10,000")
    print(f"최종 자본: ${final_capital:.2f}")
    print(f"총 수익률: {total_return:.2f}%")
    print(f"총 거래 횟수: {len(trades_df)}")
    print(f"승률: {win_rate:.1f}%")
    print(f"평균 수익: {avg_win:.2f}%")
    print(f"평균 손실: {avg_loss:.2f}%")
    
    if len(trades_df) > 0:
        print("\n=== 최근 거래 내역 ===")
        recent_trades = trades_df.tail(5)
        for _, trade in recent_trades.iterrows():
            pnl_pct = trade['pnl'] * 100
            print(f"{trade['entry_time'].strftime('%m-%d %H:%M')} ~ {trade['exit_time'].strftime('%m-%d %H:%M')}: {trade['type']} ${trade['entry_price']:.2f} → ${trade['exit_price']:.2f} ({pnl_pct:+.2f}%)")
else:
    print("거래 신호가 없어 백테스팅을 수행할 수 없습니다.")


## 9. 사용법 및 주의사항

### 사용법
1. **실시간 모니터링**: `monitor_realtime_signals(df)` 함수를 주기적으로 실행하여 새로운 매매 신호를 확인
2. **차트 분석**: `plot_trading_chart(df)` 함수로 현재 시장 상황과 신호를 시각적으로 분석
3. **백테스팅**: `backtest_strategy(df)` 함수로 과거 데이터에 대한 전략 성과를 검증

### 신호 해석
- **강력한 매수 신호 (▲)**: SQZMOM 상승 전환 + Stochastic RSI 과매도 반등 + Squeeze 해제가 동시에 발생
- **일반 매수 신호 (▲)**: 위 조건 중 2개 이상 만족
- **강력한 매도 신호 (▼)**: SQZMOM 하락 전환 + Stochastic RSI 과매수 하락이 동시에 발생
- **일반 매도 신호 (▼)**: 위 조건 중 1개 이상 만족

### 주의사항
⚠️ **이 시스템은 교육 및 연구 목적으로 제작되었습니다**
- 실제 투자에 사용하기 전에 충분한 백테스팅과 검증이 필요합니다
- 시장 상황에 따라 신호의 정확도가 달라질 수 있습니다
- 리스크 관리와 손절매 전략을 함께 사용하는 것을 권장합니다
- 과최적화(Over-fitting)에 주의하세요

### 권장 설정
- **타임프레임**: 1분봉 (단기 스캘핑용)
- **모니터링 주기**: 5분마다 신호 확인
- **포지션 크기**: 자본의 1-2% 이내
- **손절매**: 진입가 대비 0.5-1% 손실 시
- **익절**: 진입가 대비 1-2% 수익 시
